In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [11]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [12]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [13]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [14]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [16]:
import pandas as pd

train = pd.read_csv('/home/kdt-admin/lsy/significant_clean1.csv')
train=train[['clean1_result','malicious']]
test = pd.read_csv('/home/kdt-admin/lsy/significant_clean1_test.csv')
test=test[['clean1_result','malicious']]
train.sample(6)

,clean1_result,malicious
1,['은교'],0
8,"['사각', '턱', '유전자', '없다', '쌍', '커플', '태어나다', '때...",0
7,"['폭동', '진압', '병신']",1
5,"['가족', '생각']",0
9,"['댄스', '그룹']",0
0,"['석유', '화학', '오일', '현대', '뱅크']",0


In [17]:
import pandas as pd 
from ast import literal_eval

# 문자열을 리스트로 변환
def str_to_list(x):
    try:
        if type(x) == str:
        	return literal_eval(x)
        elif type(x) == list:
        	return x
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return None
train['clean1_result']=train['clean1_result'].apply(str_to_list)
test['clean1_result']=test['clean1_result'].apply(str_to_list)

In [18]:
print(type(train['clean1_result'][0]))
print(type(train['malicious'][0]))

<class 'list'>
<class 'numpy.int64'>


In [19]:
train['clean1_result'] = train['clean1_result'].apply(lambda x: ' '.join(x))
test['clean1_result'] = test['clean1_result'].apply(lambda x: ' '.join(x))
train

,clean1_result,malicious
0,석유 화학 오일 현대 뱅크,0
1,은교,0
2,김트루 디씨 네임 드다,0
3,그냥 하다 놓다 닉 구로 디지탈 사다 키 잘생기다 하다 인물 아깝다 인생 짧다,0
4,존경 롭 넹 와우,0
5,가족 생각,0
6,그렇게 생각 하 진짜,0
7,폭동 진압 병신,1
8,사각 턱 유전자 없다 쌍 커플 태어나다 때 있다,0
9,댄스 그룹,0


In [20]:
train['length'] = train['clean1_result'].apply(lambda x: len(str(x)))
test['length'] = test['clean1_result'].apply(lambda x: len(str(x)))
train = train.loc[train['length'] > 5]
test = test.loc[test['length'] > 5]
train

,clean1_result,malicious,length
0,석유 화학 오일 현대 뱅크,0,14
2,김트루 디씨 네임 드다,0,12
3,그냥 하다 놓다 닉 구로 디지탈 사다 키 잘생기다 하다 인물 아깝다 인생 짧다,0,43
4,존경 롭 넹 와우,0,9
6,그렇게 생각 하 진짜,0,11
7,폭동 진압 병신,1,8
8,사각 턱 유전자 없다 쌍 커플 태어나다 때 있다,0,26


In [21]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [22]:
class BERTDataset(Dataset):
    def __init__(self, dataframe, sent_col, label_col, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([row[sent_col]]) for _, row in dataframe.iterrows()]
        self.labels = [np.int32(row[label_col]) for _, row in dataframe.iterrows()]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return len(self.labels)


In [23]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
tok = tokenizer.tokenize

data_train = BERTDataset(train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, vocab, max_len, True, False)

/tmp/ipykernel_2220277/1666779573.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.sentences = [transform([row[sent_col]]) for _, row in dataframe.iterrows()]
/tmp/ipykernel_2220277/1666779573.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.labels = [np.int32(row[label_col]) for _, row in dataframe.iterrows()]


In [25]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [27]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [28]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [29]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/home/kdt-admin/miniconda3/envs/kobert/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [31]:
mode

NameError: name 'mode' is not defined